<h1 align='center'>Xây Dựng Mô Hình Mạng Neural Network</h1>
<h1 align='center'>Đỗ Trọng Khánh - 19521676</h1>

# Connect Driver

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# Import Libary

In [86]:
import numpy as np
import pandas as pd
from keras.layers import Dense
from keras.models import Sequential
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Prepare Data

In [75]:
data_link = '/gdrive/MyDrive/Colab Notebooks/Lập trình Python cho Máy học/Week 8/Churn_Modelling.csv'
data = pd.read_csv(data_link)

In [76]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [77]:
df = pd.DataFrame(data)
X_data = df.iloc[:, 3:-1];
y_data = df.iloc[:, -1:];

In [78]:
X_data = X_data.drop(columns = ['Geography', 'Gender'])

onehot_encoder = OneHotEncoder()

x_geography = df['Geography'].to_numpy().reshape(-1, 1)
x_geography = onehot_encoder.fit_transform(x_geography).toarray()

x_gender = df['Gender'].to_numpy().reshape(-1, 1)
x_gender = onehot_encoder.fit_transform(x_gender).toarray()

X_data = np.hstack([X_data, x_geography])
X_data = np.hstack([X_data, x_gender])

y_data = y_data.to_numpy()

# Train Test Split

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.33, random_state=42)

# Chuẩn hoá dữ liệu với Standard Scaler

In [80]:
standard_scaler = StandardScaler()

X_train = standard_scaler.fit_transform(X_train)
X_test = standard_scaler.fit_transform(X_test)

# Xây dựng mô hình Mạng Neural Network

## Define the keras model

In [81]:
model = Sequential()
model.add(Dense(12, input_dim=13, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

## Compile the keras model

In [82]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Fit the keras model on the dataset

In [83]:
model.fit(X_train, y_train, epochs=50, batch_size=10)

Epoch 1/50
670/670 [==============================] - 1s 1ms/step - loss: 0.5030 - accuracy: 0.7699
Epoch 2/50
670/670 [==============================] - 1s 1ms/step - loss: 0.4336 - accuracy: 0.8058
Epoch 3/50
670/670 [==============================] - 1s 1ms/step - loss: 0.4184 - accuracy: 0.8155
Epoch 4/50
670/670 [==============================] - 1s 1ms/step - loss: 0.4040 - accuracy: 0.8275
Epoch 5/50
670/670 [==============================] - 1s 1ms/step - loss: 0.3867 - accuracy: 0.8397
Epoch 6/50
670/670 [==============================] - 1s 1ms/step - loss: 0.3703 - accuracy: 0.8469
Epoch 7/50
670/670 [==============================] - 1s 1ms/step - loss: 0.3593 - accuracy: 0.8509
Epoch 8/50
670/670 [==============================] - 1s 1ms/step - loss: 0.3526 - accuracy: 0.8537
Epoch 9/50
670/670 [==============================] - 1s 1ms/step - loss: 0.3480 - accuracy: 0.8549
Epoch 10/50
670/670 [==============================] - 1s 1ms/step - loss: 0.3467 - accuracy: 0.8561

## Evaluate the keras model

In [85]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

104/104 [==============================] - 0s 1ms/step - loss: 0.3380 - accuracy: 0.8697
Accuracy: 86.97


# Evaluate the keras model use KFold Cross Validation

In [93]:
X_data = standard_scaler.fit_transform(X_data)

In [94]:
kfold = KFold(5, True, 1);

score_list = [];
for train, test in kfold.split(X_data, y_data):
    X_train, X_test = X_data[train], X_data[test];
    y_train, y_test = y_data[train], y_data[test];
    
    model.fit(X_train, y_train, epochs=5, batch_size=10);
    _, accuracy = model.evaluate(X_test, y_test)
    score_list.append(accuracy)

Epoch 1/5
800/800 [==============================] - 1s 1ms/step - loss: 0.8029 - accuracy: 0.7958
Epoch 2/5
800/800 [==============================] - 1s 1ms/step - loss: 0.4430 - accuracy: 0.8234
Epoch 3/5
800/800 [==============================] - 1s 1ms/step - loss: 0.4058 - accuracy: 0.8341
Epoch 4/5
800/800 [==============================] - 1s 1ms/step - loss: 0.3882 - accuracy: 0.8399
Epoch 5/5
63/63 [==============================] - 0s 1000us/step - loss: 0.3725 - accuracy: 0.8415
Epoch 1/5
800/800 [==============================] - 1s 1ms/step - loss: 0.3677 - accuracy: 0.8465
Epoch 2/5
800/800 [==============================] - 1s 1ms/step - loss: 0.3613 - accuracy: 0.8518
Epoch 3/5
800/800 [==============================] - 1s 1ms/step - loss: 0.3573 - accuracy: 0.8541
Epoch 4/5
800/800 [==============================] - 1s 1ms/step - loss: 0.3538 - accuracy: 0.8553
Epoch 5/5
63/63 [==============================] - 0s 1ms/step - loss: 0.3668 - accuracy: 0.8535
Epoch 1/5
8

In [97]:
print('Accuracy use KFold Cross Validation: %.2f' % (np.mean(score_list) * 100))

Accuracy use KFold Cross Validation: 85.25
